<div class="details">
      <div class="resultTitle">
        <h3 itemprop="name" class="booktitle">
          <a itemprop="url" href="/works/OL17930368W?edition=ia%3Aatomichabitseasy0000clea" class="results">Atomic Habits</a>
        </h3>
      </div>
        <span itemprop="author" itemscope="" itemtype="https://schema.org/Organization" class="bookauthor"> by <a href="/authors/OL7422948A">James Clear</a> </span>
        <span class="resultStats">
          <span class="resultDetails">
            <span>
              First published in 2016
            </span>—
          </span>
        </span>
        Logged 41180 times All Time
    </div>

In [74]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

In [75]:
bookData = pd.DataFrame({
    'Title': [],
    'Author': [],
    'Year Published': [],
    'Number of Logs': []
})

def processPage(url):
    global bookData

    html = requests.get(url)
    soup = None

    if html.status_code == 200:
        print("Got website Data!")
        soup = BeautifulSoup(html.text, 'html.parser')
    else:
        print("Failed to get website data")
        return
    
    bookDetails = soup.findAll('div', class_='details')
    for book in bookDetails:
        details = re.findall(r'[A-Z][^A-Z]*', book.get_text(strip=True))
        
        # Remove the last 2 elements
        del details[len(details)-1] 
        del details[len(details)-1] 

        # Clean up & process (some of) the data
        # ... remove 'by' in the books name and get the books name
        # ... and then strip out the integers to find when it was published and logged
        
        i=0
        bookName = ""
        bookNameEndIndex = 0
        for detail in details:
            if detail[len(detail)-2:len(detail)] == 'by': # Get rid of 'by' in the books name and log it
                details[i] = detail[0:len(detail)-2]
                bookName = ''.join(details[0:i+1])
                bookNameEndIndex = i

            # Get all integers
            integers = re.findall(r'\d+', detail)
            if 'published' in detail or 'Logged' in detail:
                details[i] = integers[0] 

            i+=1 # Increment iterative value
        
        # Find the authors name
        # ... removes the books name from the list
        # ... iterates through the remaining elements up until it hits an integer and stores the index in which it hit said integer
        # ... slice and join the list to get the authors name
        # ... remove the authors name from the list after storing
        author = ""
        del details[0:bookNameEndIndex+1]

        i_ = 0
        for e in details:
            if e.isdigit() == True:
                break
            i_+=1
        
        author = ''.join(details[0:i_])
        # Step 1: Add space before and after "and" (if it's part of the string without spaces)
        author = re.sub(r"(\w)(and)(\w)", r"\1 and \3", author)

        # Step 2: Apply regex to add middle initials
        author = re.sub(r"(\w+)\s+(\w+)\s+and\s+(\w+)\s+(\w+)", r"\1 T. \2 and \3 L. \4", author)

        del details[0:i_]

        # Append data
        bookData = pd.concat([bookData, pd.DataFrame({
            'Title': [bookName],
            'Author': [author],
            'Year Published': [details[0]],
            'Number of Logs': [details[1]]
        })], ignore_index=True)

for i in range(10):
    processPage(f'https://openlibrary.org/trending/forever?page={i+1}')

bookData

Got website Data!
Got website Data!
Got website Data!
Got website Data!
Got website Data!
Got website Data!
Got website Data!
Got website Data!
Got website Data!
Got website Data!


,Title,Author,Year Published,Number of Logs
0,Atomic Habits,James Clear,2016,41181
1,It Ends With Us,Colleen Hoover,2012,40568
2,The 48 Laws of Power,Robert T. Greene and Joost L. Elffers,1998,34986
3,The Subtle Art of Not Giving a F*ck,Mark Manson,2016,30182
4,Um casamento arranjado,Zana Kheiron,2019,23272
...,...,...,...,...
194,The Titan's Curse,Rick Riordan,2007,1300
195,Cien años de soledad,Gabriel García Márquez,1967,1298
196,A Wrinkle in Time,Madeleine L'Engle,1962,1296
197,"A child called ""it""",David J. Pelzer,1987,1296
